Imports

In [ ]:
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import os
import reference_solution as refsol
from scipy.fft import rfft
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import icepinn as ip

torch.set_default_dtype(torch.float64)
print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

device = ip.get_device()

In [ ]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']
NBAR = GI['Nbar']
NSTAR = GI['Nstar']

# Define t range (needs to be same as training file)
RUNTIME = 5
NUM_T_STEPS = RUNTIME + 1
#NUM_T_STEPS = RUNTIME*5 + 1

# Define initial conditions
Ntot_init = np.ones(nx_crystal)
Nqll_init = ip.get_Nqll(Ntot_init)

# Define x, t pairs for training
X_QLC = np.linspace(-L,L,nx_crystal)
t_points = np.linspace(0, RUNTIME, NUM_T_STEPS)
x, t = np.meshgrid(X_QLC, t_points)
training_set = torch.tensor(np.column_stack((x.flatten(), t.flatten()))).to(device)

In [ ]:
# Define model attributes; instantiate model
model_dimensions = torch.tensor([8, 16]).to(device)
is_sf_PINN = torch.tensor(True)
model1 = ip.IcePINN(
	num_hidden_layers=model_dimensions[0], 
	hidden_layer_size=model_dimensions[1],
	is_sf_PINN=is_sf_PINN.item()).to(device)

# Attach model attributes as buffers so they can be saved and loaded
model1.register_buffer('dimensions', model_dimensions)
model1.register_buffer('is_sf_PINN', is_sf_PINN)

# Initialize model weights with HE initialization
model1.apply(ip.init_HE)

optimizer = torch.optim.AdamW(model1.parameters(), lr=1e-5)

# Define learning rate scheduling scheme
scheduler_summed = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10000
    )

In [ ]:
MODEL_NAME = "_dummy"
print(MODEL_NAME)
print(training_set.shape)
print(ip.calc_cp_loss(model1, training_set, ip.get_misc_params(), 10, 1, 1, False).shape)
print(ip.enforced_model(training_set, model1).shape)
print(model1)


Train model

In [ ]:
ip.train_IcePINN(
    model=model1, 
    optimizer=optimizer, 
    training_set=training_set, 
    epochs=100_000, 
    name=MODEL_NAME, 
    print_every=1_000, 
    print_gradients=False,
    LR_scheduler=None)

In [ ]:
# Instantiate model2 as best version of model1
# model2 = ip.load_IcePINN(MODEL_NAME)
# model2.train()
# optimizer2 = torch.optim.AdamW(model2.parameters(), lr=1e-6)

Train again on smaller LR starting from saved checkpoint (best saved model above)

In [ ]:
# ip.train_IcePINN(
#     model=model2, 
#     optimizer=optimizer2, 
#     training_set=training_set, 
#     epochs=100_000, 
#     name=MODEL_NAME+"_round2", 
#     print_every=1_000, 
#     print_gradients=False,
#     LR_scheduler=None)